In [ ]:
from geneticalgorithm import geneticalgorithm as ga
from scipy.stats import norm
import numpy as np
import pandas as pd 
import scipy.optimize as scpo

In [ ]:
call_df = pd.read_csv('')
History = pd.read_csv('')

History['lam'] = 0 #Initiate
History['m'] = 0 #Initiate
History['v'] = 0 #Initiate

In [ ]:
def merton_jump_call(row):
    S = row.Closing_price
    K = row.stike
    T = row.nDiff / 365
    r = row.r / 100
    sigma = row.sigma
    m = row.m
    v = row.v
    lam = row.lam
    
    d1 = (np.log(S / K) + (r + (sigma ** 2) / 2) * T) / (sigma * (T ** .5))
    d2 = d1 - sigma * (T ** .5)
    C = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    
    p = 0
    for k in range(100):
        r_k = r - lam*(m-1) + (k*np.log(m) ) / T
        sigma_k = np.sqrt( sigma**2 + (k* v** 2) / T)
        k_fact = np.math.factorial(k)
        p += (np.exp(-m*lam*T) * (m*lam*T)**k / (k_fact))  * C
    
    return p 

# Non-linear LS Calibration

In [ ]:
def f_Mert(x,σ, lam, m, v):
    X, S, r, T = x
    r = r/100
    T = T/365
    d1 = (np.log(S / X) + (r + (σ ** 2) / 2) * T) / (σ * (T ** .5))
    d2 = d1 - σ * (T ** .5)
    C = S * norm.cdf(d1) - X * np.exp(-r * T) * norm.cdf(d2)
    p = 0
    for k in range(21):
        r_k = r - lam*(m-1) + (k*np.log(m) ) / T
        sigma_k = np.sqrt( σ**2 + (k* v** 2) / T)
        k_fact = np.math.factorial(k)
        p += (np.exp(-m*lam*T) * (m*lam*T)**k / (k_fact))  * C
    return p 

init_vals = [0.2, 1, -0.5, 0.45]
bounds = ( [0, 0, -np.inf, 0], [1, np.inf, 15, 2] )
params_Mert = scpo.curve_fit(f_Mert, (call_df.stike.values,call_df.Closing_price.values,call_df.r.values,call_df.nDiff.values),call_df.Close.values, p0=init_vals, bounds=bounds)

In [ ]:
call_df['sigma'] = params_Mert[0][0]
call_df['lam'] = params_Mert[0][1]
call_df['m'] = params_Mert[0][2]
call_df['v'] = params_Mert[0][3]

In [ ]:
call_df = call_df.set_index('Date')
call_df['merton'] = call_df.apply(merton_jump_call, axis=1)

In [ ]:
sigma, lam, m, v = params_Mert
size = (len(History),1)
dt = 1/365

poi_rv = np.multiply(np.random.poisson( lam*dt, size=size),
                np.random.normal(m,v, size=size)).cumsum(axis=0)
geo = np.cumsum((( -  sigma**2/2 -lam*(m  + v**2*0.5))*dt +\
                sigma*np.sqrt(dt) * \
                np.random.normal(size=size)), axis=0)
S=np.exp(geo+poi_rv)*History[0].close
S.to_csv('./Merton-stock')

# GA Calibration

In [ ]:
varbound = np.array([[0.01,1],[0.01,5],[0.01,1],[0.01,1]])

algorithm_param = {'max_num_iteration': 3000,\
                   'population_size':500,\
                   'mutation_probability':0.1,\
                   'elit_ratio': 0.01,\
                   'crossover_probability': 0.5,\
                   'parents_portion': 0.3,\
                   'crossover_type':'uniform',\
                   'max_iteration_without_improv':25}

In [ ]:
def GA_MERT(Y):
    def f(x):
    
        sigma, lam, m, v = x
        size = (len(Y),1)
        dt = 1/365
    
        poi_rv = np.multiply(np.random.poisson( lam*dt, size=size),
                         np.random.normal(m,v, size=size)).cumsum(axis=0)
        geo = np.cumsum((( -  sigma**2/2 -lam*(m  + v**2*0.5))*dt +\
                              sigma*np.sqrt(dt) * \
                              np.random.normal(size=size)), axis=0)
        S=np.exp(geo+poi_rv)*Y[0]
        S.to_csv('./Merton-stock-GA')
        return np.mean((S-Y)**2)
    model=ga(function=f,\
            dimension=4,\
            variable_type='real',\
            variable_boundaries=varbound,\
            algorithm_parameters=algorithm_param,
         convergence_curve=False,
         progress_bar=True)

    model.run()
    return model.best_variable

In [ ]:
call_df['merton-GA'] = call_df.apply(merton_jump_call, axis=1)

# Save dataframe

In [ ]:

call_df.to_csv('./Merton-input-output')

# Metrics

In [ ]:
from .utilties import utilties

In [ ]:
line1 = utilties.error_metrics(call_df['Close'], call_df['merton'])

In [ ]:
line2 = utilties.error_metrics(call_df['Close'], call_df['merton-GA'])

In [ ]:
for line in ([*line1], [*line2]):
  print('& {:.2f} & {:.2f}\% & {:.2f}\% & {:.2f}\% & {:.2f}\% & {:.2f}\% & {:.2f}\% \\\\'.format(*line))